In [4]:
# general
import json
import pandas as pd
from pprint import pprint
from time import time

# websocket
import websocket
import thread

# db
import MySQLdb
from sqlalchemy import create_engine

# http
import requests

In [6]:
SNAPSHOTS_TABLE = 'OrderBookSnapshots'
UPDATES_TABLE = 'OrderBookUpdates'

In [5]:
def get_mysql_conn():
    creds = json.load(open('../db_credentials.json', 'rb'))
    conn = MySQLdb.connect(host=creds['host'],
                           user=creds['user'],
                           passwd=creds['pwd'],
                           db=creds['db'])
    return conn


def get_sqlalchemy_engine():
    creds = json.load(open('../mysql_creds.json', 'rb'))
    engine_name = 'mysql://{user}:{pwd}@{host}/{db}'.format(**creds)
    engine = create_engine(engine_name, echo=False)
    return engine


engine = get_sqlalchemy_engine()

In [39]:
sql = 'select * from {} order by sequence desc limit 10'.format(SNAPSHOTS_TABLE)
#sql = 'select * from {} where sequence = "3423788757"'.format(SNAPSHOTS_TABLE)
snapshots = pd.read_sql(sql, con=engine, index_col='id')
snapshots.head()

,order_id,price,sequence,side,size
id,,,,,
4691229,c1013c3b-2949-4c22-b39f-e694c4302c8d,9786543.21,3425807228,ask,0.99
4681017,4c1410d5-76cc-4acd-88ca-3b9ea951a54e,2488.5,3425807228,bid,0.01
4681015,4539e9a9-cdc1-4cf7-a78f-8082595eaa19,2488.55,3425807228,bid,0.8
4681014,f96b3112-269a-43f7-bcfd-ccafa3a60887,2488.55,3425807228,bid,1.2598
4681022,16941f2d-5a9c-4a67-9651-269a1a2760bb,2486,3425807228,bid,0.044116


In [40]:
sql = 'select * from {} order by time desc limit 10'.format(UPDATES_TABLE)
updates = pd.read_sql(sql, con=engine, index_col='id')
updates.head()

,client_oid,funds,maker_order_id,new_funds,new_size,old_funds,old_size,order_id,order_type,price,product_id,reason,remaining_size,sequence,side,size,taker_order_id,time,trade_id,type
id,,,,,,,,,,,,,,,,,,,,
6542172,c59e6870-5a61-11e7-a026-193e79d4ff2a,None,None,None,None,None,None,f973522b-2c2b-414b-9c2f-3c6d6fda0338,limit,2499.96000000,BTC-USD,None,None,3425807298,sell,1.00000000,None,2017-06-26T11:22:43.240000Z,None,received
6542171,None,None,None,None,None,None,None,ea01ff92-1414-4276-a3ac-f95cd6bdf5ca,None,2500.00000000,BTC-USD,canceled,1.00000000,3425807297,sell,None,None,2017-06-26T11:22:43.208000Z,None,done
6542170,None,None,None,None,None,None,None,9198bfcb-e8e4-4ed7-9ac1-d23cb1418e2a,None,2482.21000000,BTC-USD,canceled,3.13000000,3425807296,buy,None,None,2017-06-26T11:22:43.038000Z,None,done
6542169,None,None,None,None,None,None,None,77aa9391-6b16-4e56-8149-6830113b2714,None,2502.56000000,BTC-USD,canceled,3.14000000,3425807295,sell,None,None,2017-06-26T11:22:43.000000Z,None,done
6542168,None,None,None,None,None,None,None,52a9331d-2b25-4347-b253-33384bad7951,None,2480.83000000,BTC-USD,canceled,1.87000000,3425807294,buy,None,None,2017-06-26T11:22:42.957000Z,None,done
